<a href="https://colab.research.google.com/github/AbdullatifHabiba/Face-Recognition/blob/main/LDA_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn import datasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib.image import imread
from zipfile import ZipFile
from scipy.linalg import eigh

#with ZipFile("archive.zip", "r") as folders:
# folders.extractall("data")
with ZipFile("nonface.zip", "r") as fol:
  fol.extractall("noneFace")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Creating the data

Here we read images bytes and store them in dataMatrix with size 400*10304 and create labelVector that holds the labels of the dataMatrix

In [ ]:
dataMatrix = []
labelVector = []
for i in range(1, 41):
  for j in range(1, 11):
    image = imread("data/s" + str(i) + "/" + str(j) + ".pgm")
    labelVector.append(i);
    dataMatrix.append(image.flatten())

Here we construct the training data and testing data with their labels.

In [ ]:
def devide_data(data, ratio):
  dataTraining = []
  dataTesting = []
  trainLabel = []
  testLabel = []
  for i in range(len(data)):
    if(int((i + 1) * ratio) != int(i * ratio)):
      dataTraining.append(data[i])
      trainLabel.append(labelVector[i])
    else:
      dataTesting.append(data[i])
      testLabel.append(labelVector[i])
  return dataTraining, dataTesting, trainLabel, testLabel
dataTraining, dataTesting, trainLabel, testLabel = devide_data(dataMatrix, .5)
TestDFrame = pd.DataFrame(data = dataTesting)
TrainDFrame = pd.DataFrame(data = dataTraining)

#PCA Algorithm

In [ ]:
def calculate_eigenValues_eigenVectors(data):
  centered_data = data.apply(lambda x: x-x.mean())
  cov_matrix = np.cov(centered_data.T, bias = True)
  eignValue, eignVector = eigh(cov_matrix)
  return eignValue, eignVector
eignValue, eignVector = calculate_eigenValues_eigenVectors(TrainDFrame)

In [ ]:
def PCA(alpha, data, eignValue, eignVector):
  for r in range(1, len(eignValue)):
    if sum(eignValue[-r:]) / sum(eignValue) >= alpha:
      break;
  return eignVector[-r:]

K_NN algorithm in PCA

In [ ]:
# simple classifier (first Nearest Neighbor to determine the class labels
from sklearn.neighbors import KNeighborsClassifier
def classify_by_KNN(K, trained_data, tested_data):
  Fnn = KNeighborsClassifier(n_neighbors = K)
  Fnn.fit(trained_data, trainLabel)
  tested_result = Fnn.predict(tested_data)
  ac_dif = np.array(tested_result - trainLabel[:len(tested_result)])
  # print("train accuracy", Fnn.score(trained_data, trainLabel))
  # print("test accuracy", Fnn.score(tested_data, testLabel))
  return Fnn.score(trained_data, trainLabel), Fnn.score(tested_data, testLabel)
for alpha in [.8, .85, .9, .95]:
  Projection_matrix = PCA(alpha, TrainDFrame, eignValue, eignVector)
  TrainingAfter_projection = np.dot(Projection_matrix, np.transpose(TrainDFrame))
  TestingAfter_projection = np.dot(Projection_matrix,np.transpose(TestDFrame))
  print(TestingAfter_projection.shape)
  tested_data = np.transpose(TestingAfter_projection)
  trained_data = TrainingAfter_projection.T
  # k 1 3 5 7
  K_arr = [1, 3, 5, 7]
  matrix = np.zeros((4, 2))
  i = 0
  for x in K_arr:
    matrix[i] = classify_by_KNN(x, trained_data, tested_data)
    i = i + 1
  plt.plot(K_arr, matrix[:, 1:], label = 'Testing dataset Accuracy')
  plt.plot(K_arr, matrix[:, 0:1], label = 'Training dataset Accuracy')
  plt.legend()
  plt.title('alpha = ' + str(alpha))
  plt.xlabel('k')
  plt.ylabel('Accuracy')
  plt.show()

# LDA Algorithm

In [ ]:
def LDA(dataT, size, step, trainLabel):
  means = np.zeros(((int) (size / step), len(dataT[0])))
  i = 0
  for v in range(0, size, step):
    means[i] = np.mean(dataT[v:v + step], axis = 0);
    i = i + 1
  # calculate of  𝑆𝑏 by ∑𝑛𝑘(𝜇𝑘 − 𝜇)(𝜇𝑘 − 𝜇)
  # calculate over all mean
  over_all_mean = np.mean(means, axis = 0)
  SB = np.zeros((len(means[0]), len(means[0])))
  for m in range(len(means)):
    SB = np.add(SB, step*((means[m] - over_all_mean) * (means[m] - over_all_mean).T))
  #iii.  CENTER data
  center_data = []
  if step !=5:
        center_data[0:step] = np.subtract(dataT[0:step], means[0])
        center_data[step:size+1] = np.subtract(dataT[step:size+1], means[1])
  else:
     for i in range(len(dataT)):
       c = np.array(dataT)[i,:] - np.array(means)[(trainLabel[i] - 1),:];
       center_data.insert(i, c)
  # cal S = ZT.Z
  S = np.dot(np.array(center_data).T, np.array(center_data))
  S_inv = np.linalg.inv(S)
  #cal eigen value and eigen vector
  A = np.dot(S_inv,SB)
  eigen_value, eigenvector = eigh(A, subset_by_index = [10304 - 40, 10304 - 1])
  return eigenvector

In [ ]:
# trained data
eigenvector = LDA(dataTraining, 200, 5, trainLabel)
trained_data = np.dot(dataTraining, eigenvector)

In [ ]:
# tested data
tested_data = np.dot(dataTesting, eigenvector)

K_NN Algorithm

In [ ]:
# simple classifier (first Nearest Neighbor to determine the class labels
from sklearn.neighbors import KNeighborsClassifier
def classify_by_KNN(K, trained_data, trainLabel, tested_data, testLabel):
  Fnn = KNeighborsClassifier(n_neighbors = K)
  Fnn.fit(trained_data, trainLabel)
  tested_result = Fnn.predict(tested_data)
  ac_dif = np.array(tested_result - trainLabel[:len(tested_result)])
  return Fnn.score(trained_data, trainLabel), Fnn.score(tested_data, testLabel)

In [ ]:
# k 1 3 5 7
K_arr = [1, 3, 5, 7]
matrix = np.zeros((4, 2))
i = 0
for x in K_arr:
  matrix[i] = classify_by_KNN(x,trained_data,trainLabel,tested_data,testLabel)
  i = i + 1
plt.plot(K_arr, matrix[:, 1:], label = 'Testing dataset Accuracy')
plt.plot(K_arr, matrix[:, 0:1], label = 'Training dataset Accuracy')
plt.legend()
plt.xlabel('k')
plt.ylabel('Accuracy')
plt.show()



## **Compare vs Non-Face Images**




In [ ]:
import cv2
NonFaceMatrix = [];
for i in range(150):
  image = cv2.imread("drive/MyDrive/natural_images/airplane/airplane_" + str(i).zfill(4) + ".jpg")
  image
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  res = cv2.resize(image, dsize=(92, 112), interpolation=cv2.INTER_CUBIC)
  NonFaceMatrix.append(res.flatten())
  image = cv2.imread("drive/MyDrive/natural_images/car/car_" + str(i).zfill(4) + ".jpg")
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  res = cv2.resize(image, dsize=(92, 112), interpolation=cv2.INTER_CUBIC)
  NonFaceMatrix.append(res.flatten())
  image = cv2.imread("drive/MyDrive/natural_images/cat/cat_" + str(i).zfill(4) + ".jpg")
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  res = cv2.resize(image, dsize=(92, 112), interpolation=cv2.INTER_CUBIC)
  NonFaceMatrix.append(res.flatten())
  image = cv2.imread("drive/MyDrive/natural_images/dog/dog_" + str(i).zfill(4) + ".jpg")
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  res = cv2.resize(image, dsize=(92, 112), interpolation=cv2.INTER_CUBIC)
  NonFaceMatrix.append(res.flatten())
  image = cv2.imread("drive/MyDrive/natural_images/flower/flower_" + str(i).zfill(4) + ".jpg")
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  res = cv2.resize(image, dsize=(92, 112), interpolation=cv2.INTER_CUBIC)
  NonFaceMatrix.append(res.flatten())
  image = cv2.imread("drive/MyDrive/natural_images/fruit/fruit_" + str(i).zfill(4) + ".jpg")
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  res = cv2.resize(image, dsize=(92, 112), interpolation=cv2.INTER_CUBIC)
  NonFaceMatrix.append(res.flatten())
  image = cv2.imread("drive/MyDrive/natural_images/motorbike/motorbike_" + str(i).zfill(4) + ".jpg")
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  res = cv2.resize(image, dsize=(92, 112), interpolation=cv2.INTER_CUBIC)
  NonFaceMatrix.append(res.flatten())

In [ ]:
# apply lda algrothim for this data  400 image [200 non face,200 face]
def nonFace_compare(data_size,nonface_size):
  d=(int)(nonface_size/2)
  NonFaceCompareM_train = np.array(dataTraining + NonFaceMatrix[0:d])
  NonFaceCompareL_train = np.concatenate([np.zeros(200), np.ones(d)])
  print(NonFaceCompareM_train.shape)
  eigv = LDA(NonFaceCompareM_train, data_size, d, NonFaceCompareL_train)
  newDataCompare = np.dot(NonFaceCompareM_train,eigv)
  NonFaceCompareM_test = np.array(dataTesting+NonFaceMatrix[d:nonface_size])
  NonFaceCompareL2_test = np.concatenate([np.zeros(200), np.ones(d)])
  newTestCompare = np.dot(NonFaceCompareM_test, eigv)
  return  newDataCompare,NonFaceCompareL_train,newTestCompare,NonFaceCompareL2_test

In [ ]:

K_arr = [1, 3, 5, 7]
nonface_images=[400,600,800,1000]
matrix1 = np.zeros((4, 2))
res=[]

for j in nonface_images:
  i = 0
  for x in K_arr:
    matrix1[i] = classify_by_KNN(x,nonFace_compare((int)((400+j)/2),j))
    i = i + 1

  plt.plot(K_arr, matrix1[:, 1:], label = 'Testing dataset Accuracy')
  plt.plot(K_arr, matrix1[:, 0:1], label = 'Training dataset Accuracy')
  plt.legend()
  plt.xlabel('k')
  plt.ylabel('Accuracy')
  plt.show()
  res.append(matrix1)
res
plt.scatter(nonface_images,np.mean(matrix1,axis=1))



#Second bonus part

In [ ]:
from sklearn.decomposition import IncrementalPCA
X = (dataTraining);
no_of_batches = 1;
incremental_pca = IncrementalPCA(n_components = 170)
for batch in np.array_split(X, no_of_batches):
  incremental_pca.fit(batch)
finalTraining = incremental_pca.transform(X)
print(finalTraining.shape)

In [ ]:
X = (dataTesting);
no_of_batches = 1;
incremental_pca = IncrementalPCA(n_components = 170)
for batch in np.array_split(X, no_of_batches):
  incremental_pca.fit(batch)
finalTesting = incremental_pca.transform(X)
print(finalTesting.shape)

In [ ]:
tested_data = finalTesting
trained_data = finalTraining
# k 1 3 5 7
K_arr = [1, 3, 5, 7]
matrix = np.zeros((4, 2))
i = 0
for x in K_arr:
  matrix[i] = classify_by_KNN(x, trained_data, trainLabel, tested_data, testLabel)
  i = i + 1
print(matrix)
plt.plot(K_arr, matrix[:, 1:], label = 'Testing dataset Accuracy')
plt.plot(K_arr, matrix[:, 0:1], label = 'Training dataset Accuracy')
plt.legend()
plt.xlabel('k')
plt.ylabel('Accuracy')
plt.show()

#Shrinkage linear discriminant analysis

Shrinkage linear discriminant analysis is a variant of normal one but uses shrinkage regulator to estimate the covariance matrix, to increase stability and solve overfitting problems in normal LDA

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
X = dataMatrix
y = labelVector
X_train, X_test,\
    y_train, y_test = train_test_split(X, y, test_size = 0.3)
estimator = LinearDiscriminantAnalysis(solver = 'eigen', shrinkage = 'auto')
estimator.fit(X_train, y_train)
y_pred = estimator.predict(X_test)
print(estimator.score(X_test, y_test))